### IMPORTS

In [54]:
import sys
path = 'C:\\Users\\santiago.curat\\Pandas\\PEYA'
sys.path.insert(0,path)

In [55]:
import pandas as pd
import numpy as np
import calendar
import datetime
from dateutil.relativedelta import relativedelta
from Roster_FEHGRA import *
from gspread_pandas import Spread, conf

### CREDENCIALES Y CONEXIONES

In [56]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### GOOGLE SHEETS

In [81]:
# Me traigo los impactos
sheet_id = '18TKPCJy746jqLvD6u2fkuOYkgpXVYoOUe5cIU-LTgaI'
wks_name = 'Analisis Post'
sheet = Spread(sheet_id, wks_name, config=cred)
base = sheet.sheet_to_df(index=0,header_rows=1)

In [82]:
# Limpio las filas vacias
base = base[base['Grids'] != ''].copy()
#Filtro columnas
base = base[['Grids','Ejecución Impacto','Impacto','Comi Vieja 2','Comision Nueva','Camara']].copy()
base.columns = ['Grids','Ejecucion','Impacto','Comision Vieja','Comision Nueva','Camara']

In [83]:
# Cambio el formato de las fechas
base['Ejecucion'] = [x.replace('/', '-') for x in base['Ejecucion']]
base['Ejecucion'] = base['Ejecucion'].apply(lambda x: str(datetime.date.today().replace(day=1))[:-3] if x == '' else x)
# Cambio el formato a las comisiones
base['Comision Vieja'] = [float(x.replace(',','.').replace('%','')) for x in base['Comision Vieja']]
# Armo lista de Grids
grids = str(base['Grids'].to_list()).replace('[','(').replace(']',')')

### CONSTANTES

In [85]:
inicio = '2021-04-01'
fin = '2021-04-30'

### QUERIES

In [86]:
q_p = '''WITH orders_table AS (
    SELECT o.restaurant.id AS id,
           SUBSTR(CAST(o.registered_date AS STRING),1,7) AS month,
           SUM(CASE WHEN o.with_logistics THEN o.total_amount + o.discount_paid_by_company + o.shipping_amount - o.shipping_amount_no_discount
               ELSE o.total_amount + o.shipping_amount + o.discount_paid_by_company END) AS income,
           SUM(o.commission_amount) AS Revenue
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    WHERE o.country_id = 3
          AND o.registered_date >= DATE('2020-12-01')
          AND o.order_status = 'CONFIRMED'
    GROUP BY 1,2)
SELECT p.salesforce_id AS Grids,
       ot.month AS Month,
       CASE WHEN p.is_online THEN 'Si' ELSE 'No' END AS Online,
       CASE WHEN p.is_logistic THEN 'Si' ELSE 'No' END AS Logistic,
       IFNULL(p.billingInfo.sap_id,'-') AS SAP_Id,
       IFNULL(p.billingInfo.rut,'-') AS CUIT,
       p.city.name AS City,
       a.area_name AS Area,
       p.billingInfo.partner_commission AS Commission,
       ot.income AS Income,
       ot.revenue AS Revenue
FROM `peya-bi-tools-pro.il_core.dim_partner` AS p
LEFT JOIN orders_table AS ot ON p.partner_id = ot.id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
WHERE p.country_id = 3
      AND p.salesforce_id IN {0}'''.format(grids)

q_resto = '''SELECT p.city.name AS City,
       a.area_name AS Area,
       SUM(CASE WHEN o.with_logistics THEN o.total_amount + o.discount_paid_by_company + o.shipping_amount - o.shipping_amount_no_discount ELSE 0 END) AS Income_Log,
       SUM(CASE WHEN o.with_logistics = FALSE THEN o.total_amount + o.shipping_amount + o.discount_paid_by_company ELSE 0 END) AS Income_Mktp,
       SUM(CASE WHEN o.with_logistics THEN o.commission_amount ELSE 0 END) AS Revenue_Log,
       SUM(CASE WHEN o.with_logistics= FALSE THEN o.commission_amount ELSE 0 END) AS Revenue_Mktp
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON o.restaurant.id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
WHERE p.country_id = 3
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.salesforce_id NOT IN {1}
      AND o.order_status = 'CONFIRMED'
GROUP BY 1,2'''.format(inicio,fin,grids)

In [51]:
# Descargo la data
hue_p = pd.io.gbq.read_gbq(q_p, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 10626/10626 [00:02<00:00, 4541.42rows/s]


In [52]:
hue_resto = pd.io.gbq.read_gbq(q_resto, project_id='peya-argentina', dialect='standard')

Downloading: 100%|████████████████████████████████████████████████████████████████| 886/886 [00:01<00:00, 655.23rows/s]


In [61]:
# Copio las bases
partners = hue_p.copy()
resto = hue_resto.copy()

### FUNCIONES

In [63]:
def comisiones(i):
    if i['Comision Vieja'] == 0:
        bandera = 0
        for j in tuplas_com:
            if i['Grids'] == j[0]:
                val = j[1]
                bandera = 1
        if bandera == 0:
            val = 'Error'
    else:
        val = i['Comision Vieja']
    return val

def incomes_revenues(i):
    bandera = 0
    for j in tuplas_inc:
        if i['Grids'] == j[1] and str(j[0]) == (i['Ejecucion'] - pd.DateOffset(months=1)).strftime('%Y-%m'):
            income = j[2]
            revenue = j[3]
            bandera = 1
    if bandera == 0:
        income = 0
        revenue = 0
    return pd.Series([income,revenue])

def nuevas_comisiones(i):
    if i['Comision Nueva'] == '-':
        if i['Logistic'] == 'Si':
            val = 18
        else:
            val = 10
    else:
        val = i['Comision Nueva']
    return val

### TRABAJO

In [87]:
# Creo la lista de tuplas para comisiones
tuplas_com = list(partners[['Grids','Commission']].drop_duplicates().to_records(index=False))

In [65]:
# Coloco la comision a los partners faltantes
base['Comision Vieja'] = base.apply(comisiones,axis=1)
base = base[base['Comision Vieja'] != 'Error'].copy()

In [66]:
# Doy formato a la fecha
partners['Month'] = pd.to_datetime(partners['Month'], format='%Y-%m').dt.strftime('%Y-%m')
base['Ejecucion'] = pd.to_datetime(base['Ejecucion'], format='%Y-%m')

In [89]:
# Creo una lista de tuplas para Income
tuplas_inc = list(partners[['Month','Grids','Income','Revenue']].to_records(index=False))

In [68]:
# Coloco el Income LM
base[['Income LM','Revenue LM']] = base.apply(incomes_revenues,axis=1)
# Cabmio el formato al mes
base['Ejecucion'] = pd.to_datetime(base['Ejecucion'], format='%Y-%m-%d').dt.strftime('%Y-%m')

In [69]:
# Agrego Feudo y Reino a partners
partners['Feudo'] = partners.apply(feudos,axis=1)
partners['Reino'] = partners.apply(reinos,axis=1)

In [70]:
# Agrego Online y Logistic a la Base
final_base = base.merge(partners[['Grids','Online','Logistic','Feudo','Reino','SAP_Id','CUIT']].drop_duplicates(),on=['Grids'],how='left')

In [71]:
# Coloco la comision nueva estimada a los partners nuevos
final_base['Comision Nueva'] = final_base.apply(nuevas_comisiones,axis=1)

In [72]:
# Coloco el nuevo Revenue
final_base['Income LM'] = final_base['Income LM'].astype(float)
final_base['Comision Nueva'] = final_base['Comision Nueva'].astype(float)
final_base['Revenue Nuevo'] = final_base['Income LM'] * (final_base['Comision Nueva']/100)
final_base['Revenue Nuevo'].replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [73]:
# Coloco Feudo y Reino al resto
resto['Feudo'] = resto.apply(feudos,axis=1)
resto['Reino'] = resto.apply(reinos,axis=1)

In [74]:
# Creo una PT del resto
values = ['Income_Log','Income_Mktp','Revenue_Log','Revenue_Mktp']
resto[values] = resto[values].astype(float)
pt_resto = resto.pivot_table(index=['Reino','Feudo'],values=values,aggfunc='sum',fill_value=0).reset_index()

In [75]:
# Arreglo el formato de numeros a Final_Base
final_base['Comision Vieja'] = [str(x).replace('.', ',') for x in final_base['Comision Vieja']]
final_base['Comision Nueva'] = [str(x).replace('.', ',') for x in final_base['Comision Nueva']]
final_base['Income LM'] = [str(x).replace('.', ',') for x in final_base['Income LM']]
final_base['Revenue LM'] = [str(x).replace('.', ',') for x in final_base['Revenue LM']]
final_base['Revenue Nuevo'] = [str(x).replace('.', ',') for x in final_base['Revenue Nuevo']]
# Arreglo el formato de numeros a PT_Resto
pt_resto['Income_Log'] = [str(x).replace('.', ',') for x in pt_resto['Income_Log']]
pt_resto['Income_Mktp'] = [str(x).replace('.', ',') for x in pt_resto['Income_Mktp']]
pt_resto['Revenue_Log'] = [str(x).replace('.', ',') for x in pt_resto['Revenue_Log']]
pt_resto['Revenue_Mktp'] = [str(x).replace('.', ',') for x in pt_resto['Revenue_Mktp']]

In [78]:
# Ordeno las columnas de Final_Base
cols = ['Grids','Ejecucion','Impacto','Comision Vieja','Comision Nueva','Camara','Income LM','Revenue LM','Online','Logistic',
        'Feudo','Reino','Revenue Nuevo','SAP_Id','CUIT']
final_base = final_base[cols].copy()

### CARGA

In [80]:
# Carga Final Impactos
sheet_id = '18TKPCJy746jqLvD6u2fkuOYkgpXVYoOUe5cIU-LTgaI'
wks_name = 'Analisis Economico Impactos'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(final_base, index=False, sheet=wks_name, replace=True)

In [34]:
# Carga Final Resto
sheet_id = '18TKPCJy746jqLvD6u2fkuOYkgpXVYoOUe5cIU-LTgaI'
wks_name = 'Analisis Economico Resto' 
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(pt_resto, index=False, sheet=wks_name, replace=True)

{'spreadsheetId': '18TKPCJy746jqLvD6u2fkuOYkgpXVYoOUe5cIU-LTgaI',
 'replies': [{'findReplace': {'valuesChanged': 96,
    'rowsChanged': 24,
    'sheetsChanged': 1,
    'occurrencesChanged': 96}}]}